# Lib

In [301]:
%load_ext autoreload
%autoreload 2

import pandas as pd

import warnings

## ignore warning log in pandas
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib
import sys

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")

from utils import utils_ml, parameters, metrics_evaluation, Pipeline_functions


from ipywidgets import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [302]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

25/11/2022  01:34    <DIR>          .
23/11/2022  00:25    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
21/11/2022  23:51         1.007.073 1.EA.ipynb
25/11/2022  01:27            74.828 2.Feature_engineering.ipynb
25/11/2022  01:27            28.364 3.Stability_var.ipynb
26/11/2022  23:56           142.084 4.Modeling.ipynb
25/11/2022  01:32           156.224 5.Pipeline.ipynb
25/11/2022  01:32           156.224 6.Pipeline_SPARK.ipynb
24/11/2022  23:44    <DIR>          catboost_info
               6 arquivo(s)      1.564.797 bytes
               4 pasta(s)   356.273.971.200 bytes dispon�veis


# Reading

In [303]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [304]:
!dir .\\..\data

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\data

24/11/2022  23:37    <DIR>          .
23/11/2022  00:25    <DIR>          ..
10/08/2021  12:08        26.677.890 dados.csv
22/11/2022  23:52        13.834.060 EA.pbix
22/11/2022  22:34            39.018 iv_woe_by_week.parquet
09/11/2022  10:35            54.091 MySQL_localhost_connection.pbix
24/11/2022  23:31         3.891.056 test_dataset.parquet
24/11/2022  23:31         8.711.592 train_dataset.parquet
25/11/2022  01:12        14.493.356 train_test_dataset_tranformed.parquet
24/11/2022  23:31         2.094.822 validation_dataset.parquet
               8 arquivo(s)     69.795.885 bytes
               2 pasta(s)   356.273.709.056 bytes dispon�veis


In [305]:
def create_woe_columns(df):

    ## criando uma copia
    dff = df.copy()
    ## Selecionando as colunas para serem usadas
    colunas_orig_and_week = utils_ml.exclude_words( dff.columns.tolist(), ["cat","bin","WoE","produto","fecha","data","hora","week","fraude"] ) + ['categoria_produto']
    ## Iterando para adicionar as colunas WoE
    for n,variable in enumerate(colunas_orig_and_week, start=1):

        optb = joblib.load(f".\..\src\\features\\pipeline_binning__{variable}__.pkl")

        dff = utils_ml.encode_woe_var_transform(dff, variable, optb )


    return dff


In [306]:
## test-train dataset
df = pd.read_parquet(f"{parameters.data_path}\\train_test_dataset_tranformed.parquet")
df['data'] = pd.to_datetime(df['data'])

## validation Dataset
df_validation = pd.read_parquet(f"{parameters.data_path}\\validation_dataset.parquet")
df_validation['data'] = pd.to_datetime(df_validation['data'])
## Creating WoE columns
df_validation = create_woe_columns(df_validation)

In [307]:
features_bins_woe = utils_ml.list_subset_words( df.columns.tolist(), ["bin"])
## Excluding variable 'score' (score of Meli model) and 'o_obj' (with 72.5% of missing values))
features_bins_woe = utils_ml.exclude_words(features_bins_woe, ["score","o_obj"])
features_bins_woe

['n_boolean_bins',
 'p_boolean_bins',
 'categoria_produto_bins',
 'pais_bins',
 'a_int_bins',
 'b_float_bins',
 'c_float_bins',
 'd_float_bins',
 'e_float_bins',
 'f_float_bins',
 'h_float_bins',
 'k_float_bins',
 'l_float_bins',
 'm_float_bins',
 'monto_bins']

# Model

## Logistic regression

In [308]:
df_2_model = df[features_bins_woe + ["fraude","data"]]

### Dumming features

In [309]:
# from sklearn import preprocessing

# le = preprocessing.LabelEncoder()

# cat_vars = features_bins_woe

# df_2_model["pais_hot_encode"] = le.fit_transform(df_2_model['pais_bins'])


# X = data.iloc[:, 0:4]
# y = data.iloc[:, 5]

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

# classifier = LogisticRegression()
# classifier.fit(X_train, y_train)

# df_2_model[["pais_hot_encode",'pais_bins']].value_counts()

In [310]:
def transform_2_dummies( df , label = "fraude", drop_first = True):

    features = utils_ml.list_subset_words( df.columns.tolist(), ["bin"])
    features = utils_ml.exclude_words(features, ["score","o_obj","data"])

    ls_df_dummies = []

    for var in features:

        ## creating dummy to variable
        df_dummy = pd.get_dummies( df[var] , prefix=var, drop_first = drop_first)
        ls_df_dummies.append(df_dummy)

    try:
        ## Quando exista o label e o campo data
        df_2_model_dum = pd.concat(ls_df_dummies + [ df[[label,'data']] ], axis=1)
    except:
        try:
            ## Quando exista o campo label
            df_2_model_dum = pd.concat(ls_df_dummies + [ df[[label]] ], axis=1)
        except:
            ## Quando não exista o campo label nem o campo data
            df_2_model_dum = pd.concat(ls_df_dummies , axis=1)


    return df_2_model_dum

In [311]:
df_2_model_dum = transform_2_dummies( df_2_model ,  label = "fraude")

In [312]:
def transform_2_model(df ,  label ="fraude", cols_model = utils_ml.exclude_words( df_2_model_dum.columns.tolist(),['data']) ):

    # print(cols_model)

    ## tirando a coluna label caso esteja na lista
    cols_model = utils_ml.exclude_words(cols_model, [label] )
    ## Transformando para dummies
    df_marco_model_dum = transform_2_dummies(df = df, label ="fraude")

    ## Preenchendo colunas faltantes
    dummies_faltantes = list(   set(cols_model) - set(df_marco_model_dum.columns.to_list()) )

    if len(dummies_faltantes)>0:

        for dummi_falt in dummies_faltantes:

            df_marco_model_dum[dummi_falt] = 0

    try:
        ## Quando o label exista no dataFrame
        df_2_model = df_marco_model_dum[cols_model+[label]]

    except:
        ## Quando não exista o campo label
        df_2_model = df_marco_model_dum[cols_model]


    return df_2_model

### Train and test samples

In [313]:
label = "fraude"

X_train, X_test, y_train, y_test = metrics_evaluation.train_test_split_2_model( df_2_model_dum, label, test_size = 0.3, random_state = 0, by_date = False, date_field = "data", n_days_test = 5 )

In [314]:
X_train.shape[0]/df_2_model_dum.shape[0]

0.6999992272801032

### Trainning

In [315]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV, StratifiedKFold

# lr = LogisticRegression(random_state=0,solver='newton-cg', fit_intercept=True, penalty='l2' )

# #Setting the range for class weights
# weights = np.linspace(0.0,0.99,200)

# #Creating a dictionary grid for grid search
# param_grid = {'class_weight': [{0:x, 1:1.0-x} for x in weights]}

# #Fitting grid search to the train data with 5 folds
# gridsearch = GridSearchCV(estimator= lr, 
#                           param_grid= param_grid,
#                           cv=StratifiedKFold(), 
#                           n_jobs=-1, 
#                           scoring='recall', 
#                           verbose=2).fit(X_train, y_train)

## Ploting the score for different values of weight
# sns.set_style('whitegrid')
# plt.figure(figsize=(12,8))
# weigh_data = pd.DataFrame({ 'score': gridsearch.cv_results_['mean_test_score'], 'weight': (1-weights)})
# sns.lineplot(weigh_data,  x='weight', y='score')
# plt.xlabel('Weight for class 1')
# plt.ylabel('F1 score')
# plt.xticks([round(i/10,1) for i in range(0,11,1)])
# plt.title('Scoring for different class weights', fontsize=24)

In [316]:
# LogisticRegression??

In [317]:
clf = LogisticRegression( random_state=0, solver='saga',l1_ratio = 0.1, fit_intercept=True, penalty='elasticnet', class_weight = "balanced" )

model = clf.fit(X_train, y_train )

c:\Users\caanp\miniconda3\envs\venv_ml\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [318]:
colunas_orig = utils_ml.exclude_words( df.columns.tolist(), ["bin","cat","WoE","fecha",'data','hora'] ) + ['categoria_produto']
colunas_orig, len(colunas_orig)

(['a_int',
  'b_float',
  'c_float',
  'd_float',
  'e_float',
  'f_float',
  'pais',
  'h_float',
  'produto',
  'k_float',
  'l_float',
  'm_float',
  'n_boolean',
  'o_obj',
  'p_boolean',
  'monto',
  'score',
  'fraude',
  'categoria_produto'],
 19)

In [319]:
df_ivs_ls = []

for variable in colunas_orig:

    label = "fraude"
    dtype_ = 'categorical' if df[variable].dtype=='O' or df[variable].dtype=='category' else 'numerical'

    df_iv = utils_ml.WOE_IV(df = df , variable = variable, dtype_ = dtype_ , label = label, monotonic= True, optimal_binning=True)[[ 'variable' , 'IV_var', 'KS']].drop_duplicates()

    df_ivs_ls.append(df_iv)

pd.concat(df_ivs_ls).sort_values("IV_var", ascending=False)

,variable,IV_var,KS
0,produto,6.575706,89.262752
0,categoria_produto,1.541493,40.920152
0,o_obj,1.111551,45.338385
0,score,0.989984,42.673514
0,f_float,0.443467,27.653106
0,l_float,0.428472,25.054252
0,n_boolean,0.363338,22.847129
0,m_float,0.341464,23.071558
0,p_boolean,0.246970,24.429978
0,d_float,0.170680,16.498234


In [320]:
coef_df = pd.DataFrame( {"Cols": model.feature_names_in_,  "coef": model.coef_[0] })
coef_df.sort_values("coef", ascending=False)

,Cols,coef
11,categoria_produto_bins_10,7.788653
8,categoria_produto_bins_7,5.836282
9,categoria_produto_bins_8,5.835362
7,categoria_produto_bins_6,5.475225
4,categoria_produto_bins_3,5.202218
10,categoria_produto_bins_9,5.028799
5,categoria_produto_bins_4,4.954126
6,categoria_produto_bins_5,4.001895
2,categoria_produto_bins_1,3.490188
3,categoria_produto_bins_2,3.213162


In [321]:
df_2_model_dum['probability'] = model.predict_proba(df_2_model_dum.drop(['fraude','data'], axis=1))[:,1]

## Catboost

In [322]:
import catboost as cb

### Train and test samples

In [391]:
def transform_2_cat_model(df):

    cat_features = [x for x in utils_ml.list_subset_words( df.columns.tolist(), ['cat']) if "WoE" not in x and "bin" not in x and x!='categoria_produto']
    cat_features = utils_ml.exclude_words(cat_features, ["score","o_obj","data"])

    try:
        df_2_model_cat = df[ cat_features + ["fraude"] ]
    except:
        df_2_model_cat = df[ cat_features ]


    return df_2_model_cat

In [392]:
cat_features = [x for x in utils_ml.list_subset_words( df.columns.tolist(), ['cat']) if "WoE" not in x and "bin" not in x and x!='categoria_produto']
## Excluindo os campos de data, score e o_obj
cat_features = utils_ml.exclude_words(cat_features, ["score","o_obj","data"])


df_2_model_cat = df[cat_features + ["fraude","data"]]

label = "fraude"

X_train_cat, X_test_cat, y_train_cat, y_test_cat = metrics_evaluation.train_test_split_2_model( df_2_model_cat, label, test_size = 0.3, random_state = 0, by_date = False, date_field = "data", n_days_test =5 )

In [393]:
cat_features

['n_boolean_cat',
 'p_boolean_cat',
 'categoria_produto_cat',
 'pais_cat',
 'a_int_cat',
 'b_float_cat',
 'c_float_cat',
 'd_float_cat',
 'e_float_cat',
 'f_float_cat',
 'h_float_cat',
 'k_float_cat',
 'l_float_cat',
 'm_float_cat',
 'monto_cat']

### Trainning

In [394]:
model_cat_def = cb.CatBoostClassifier(
                           l2_leaf_reg = 3.0,
                           scale_pos_weight = 19,
                           verbose=False,
                           thread_count = 8,
                           random_seed = 0
                          )


modelo_cat = model_cat_def.fit(X_train_cat , y_train_cat, cat_features =  cat_features  )

In [395]:
coef = pd.DataFrame( {"features":modelo_cat.feature_names_,"coef":modelo_cat.feature_importances_})
coef = coef.sort_values("coef", ascending=False).reset_index(drop=True).query("coef>0")
coef

,features,coef
0,categoria_produto_cat,35.386707
1,f_float_cat,12.462671
2,l_float_cat,6.844596
3,e_float_cat,6.192568
4,monto_cat,6.174507
5,m_float_cat,5.606291
6,d_float_cat,4.417249
7,h_float_cat,4.282970
8,b_float_cat,4.063897
9,pais_cat,3.662281


In [396]:
# pd.concat(df_ivs_ls).sort_values("IV_var", ascending=False)

In [397]:
df_2_model_cat['probability'] = modelo_cat.predict_proba(df_2_model_cat.drop(['fraude','data'], axis=1))[:,1]

## Comparing with Meli model

> **DataSet Train_test**

In [362]:
label="fraude"
quantil=10

print(f"\t\t*************************************************************************************", )
print(f"\t\t*********************** DEV DATASET: TRAIN + TEST ************************************",)
print(f"\t\t*************************************************************************************", "\n\n")

print(f"\t\t========================== Modelo MELI =====================================")
display(utils_ml.metric_evaluation(df, 'score', label, quantil))
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df, label, 'score'))


print("\n\n",f"\t\t========================== Modelo RG =====================================")
display( utils_ml.metric_evaluation(df_2_model_dum, 'probability', label, quantil) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_2_model_dum, label, 'probability'))


print("\n\n",f"\t\t========================== Modelo Catboost =====================================")
display( utils_ml.metric_evaluation(df_2_model_cat, 'probability', label, quantil) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_2_model_cat, label, 'probability'))

		*************************************************************************************
		*********************** DEV DATASET: TRAIN + TEST ************************************
		************************************************************************************* 


		========================== Modelo MELI =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,12942,"[0, 8]",368,12574,6560,122853,0,0,5.609756,10.234996,5.609756,10.234996,42.087631,1.000000,0.050690,0.000000,0.096490,0.210725
1,12941,"[8, 18]",612,12329,6192,110279,12574,368,9.329268,10.035571,14.939024,20.270567,42.087631,0.943902,0.053163,0.102350,0.100658,0.216942
2,12941,"[18, 28]",404,12537,5580,97950,24903,980,6.158537,10.204879,21.097561,30.475446,42.087631,0.850610,0.053897,0.202706,0.101372,0.214996
3,12941,"[28, 39]",43,12898,5176,85413,37440,1384,0.655488,10.498726,21.753049,40.974172,42.087631,0.789024,0.057137,0.304754,0.106558,0.221520
4,12942,"[39, 49]",89,12853,5133,72515,50338,1427,1.356707,10.462097,23.109756,51.436269,42.087631,0.782470,0.066106,0.409742,0.121912,0.247045
5,12941,"[49, 59]",166,12775,5044,59662,63191,1516,2.530488,10.398606,25.640244,61.834876,42.087631,0.768902,0.077953,0.514363,0.141554,0.277307
6,12941,"[59, 69]",289,12652,4878,46887,75966,1682,4.405488,10.298487,30.045732,72.133363,42.087631,0.743598,0.094234,0.618349,0.167270,0.312672
7,12941,"[69, 79]",710,12231,4589,34235,88618,1971,10.823171,9.955801,40.868902,82.089163,42.087631,0.699543,0.118200,0.721334,0.202230,0.352653
8,12941,"[79, 89]",1157,11784,3879,22004,100849,2681,17.637195,9.591951,58.506098,91.681115,42.087631,0.591311,0.149867,0.820892,0.239127,0.372101


AUROC , AUPR:  (0.7244964088550802, 0.20112328214314518)


 		========================== Modelo RG =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,12942,"[2.80308733614377e-08, 2.512603063334869e-05]",19,12923,6560,122853,0,0,0.289634,10.519076,0.289634,10.519076,41.493489,1.000000,0.050690,0.000000,0.096490,0.210725
1,12941,"[2.513711136787876e-05, 0.00013761718355874102]",46,12895,6541,109930,12923,19,0.701220,10.496284,0.990854,21.015360,41.493489,0.997104,0.056160,0.105191,0.106331,0.229169
2,12941,"[0.00013764465742968407, 0.0006243204621440696]",145,12796,6495,97035,25818,65,2.210366,10.415700,3.201220,31.431060,41.493489,0.990091,0.062735,0.210154,0.117994,0.250250
3,12941,"[0.0006244784523560838, 0.00217798784079298]",300,12641,6350,84239,38614,210,4.573171,10.289533,7.774390,41.720593,41.493489,0.967988,0.070097,0.314311,0.130727,0.271765
4,12942,"[0.0021780402718576146, 0.0060980621747079]",370,12572,6050,71598,51255,510,5.640244,10.233368,13.414634,51.953961,41.493489,0.922256,0.077916,0.417206,0.143692,0.291179
5,12941,"[0.0060983869786335595, 0.015695568028529804]",501,12440,5680,59026,63827,880,7.637195,10.125923,21.051829,62.079884,41.493489,0.865854,0.087782,0.519540,0.159403,0.312273
6,12941,"[0.01569718594197392, 0.04037280730729072]",627,12314,5179,46586,76267,1381,9.557927,10.023361,30.609756,72.103245,41.493489,0.789482,0.100048,0.620799,0.177591,0.331966
7,12941,"[0.04037679346992429, 0.11579265098828662]",842,12099,4552,34272,88581,2008,12.835366,9.848355,43.445122,81.951601,41.493489,0.693902,0.117247,0.721032,0.200599,0.349809
8,12941,"[0.11579386787576734, 0.3920512666974438]",1235,11706,3710,22173,100680,2850,18.826220,9.528461,62.271341,91.480062,41.493489,0.565549,0.143337,0.819516,0.228709,0.355889


AUROC , AUPR:  (0.786665768185575, 0.17726253691300714)


 		========================== Modelo Catboost =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,12942,"[0.0006228670348677954, 0.018657679984406053]",1,12941,6560,122853,0,0,0.015244,10.533727,0.015244,10.533727,57.455025,1.000000,0.050690,0.000000,0.096490,0.210725
1,12941,"[0.018657825586940235, 0.03126129562510869]",13,12928,6559,109912,12941,1,0.198171,10.523146,0.213415,21.056873,57.455025,0.999848,0.056314,0.105337,0.106624,0.229800
2,12941,"[0.031265485231785316, 0.10453539935500601]",39,12902,6546,96984,25869,14,0.594512,10.501982,0.807927,31.558855,57.455025,0.997866,0.063228,0.210569,0.118921,0.252215
3,12941,"[0.1045394444533504, 0.22227273961403962]",59,12882,6507,84082,38771,53,0.899390,10.485702,1.707317,42.044557,57.455025,0.991921,0.071830,0.315589,0.133959,0.278484
4,12942,"[0.22228142856200814, 0.29304725437208656]",154,12788,6448,71200,51653,112,2.347561,10.409188,4.054878,52.453746,57.455025,0.982927,0.083041,0.420446,0.153145,0.310334
5,12941,"[0.2930618126755099, 0.36463109335148935]",262,12679,6294,58412,64441,266,3.993902,10.320464,8.048780,62.774210,57.455025,0.959451,0.097271,0.524537,0.176634,0.346030
6,12941,"[0.36464995175940784, 0.44925094552111067]",486,12455,6032,45733,77120,528,7.408537,10.138133,15.457317,72.912342,57.455025,0.919512,0.116527,0.627742,0.206841,0.386642
7,12941,"[0.44926061881238155, 0.5615937380356115]",711,12230,5546,33278,89575,1014,10.838415,9.954987,26.295732,82.867329,57.455025,0.845427,0.142850,0.729123,0.244403,0.426196
8,12941,"[0.5616049107976375, 0.7108235619203292]",1336,11605,4835,21048,101805,1725,20.365854,9.446249,46.661585,92.313578,57.455025,0.737043,0.186802,0.828673,0.298061,0.463807


AUROC , AUPR:  (0.8760432301056608, 0.3666664558390568)


>> **test sample**

In [404]:
label="fraude"
quantil=10

print(f"\t\t*************************************************************************************", )
print(f"\t\t*********************************** TEST DATASET ************************************",)
print(f"\t\t*************************************************************************************","\n\n")


print(f"\t\t========================== Modelo MELI =====================================")
test_2_meli = X_test_cat[['monto_cat']].merge( df[['fraude','score']], left_index=True, right_index=True ).drop(['monto_cat'], axis =1)

display(utils_ml.metric_evaluation( test_2_meli, 'score', label, quantil))
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR( test_2_meli, label, 'score'))


print("\n\n",f"\t\t========================== Modelo RG =====================================")
df_test = pd.DataFrame()
df_test['fraude'] = y_test
df_test['probability'] = model.predict_proba(X_test)[:,1]

display( utils_ml.metric_evaluation(df_test, 'probability', label, quantil) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_test, label, 'probability'))


print("\n\n",f"\t\t========================== Modelo Catboost =====================================")
df_test_cat = pd.DataFrame()
df_test_cat['fraude'] = y_test_cat
df_test_cat['probability'] = modelo_cat.predict_proba(X_test_cat)[:,1]


display( utils_ml.metric_evaluation(df_test_cat, 'probability', label, quantil) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_test_cat, label, 'probability'))

		*************************************************************************************
		*********************************** TEST DATASET ************************************
		************************************************************************************* 


		========================== Modelo MELI =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,3883,"[0, 8]",110,3773,1968,36856,0,0,5.589431,10.237139,5.589431,10.237139,42.853675,1.000000,0.050690,0.000000,0.096490,0.210725
1,3882,"[8, 18]",187,3695,1858,33083,3773,110,9.502033,10.025505,15.091463,20.262644,42.853675,0.944106,0.053175,0.102371,0.100680,0.216990
2,3882,"[18, 29]",105,3777,1671,29388,7468,297,5.335366,10.247992,20.426829,30.510636,42.853675,0.849085,0.053801,0.202626,0.101190,0.214610
3,3883,"[29, 38]",7,3876,1566,25611,11245,402,0.355691,10.516605,20.782520,41.027241,42.853675,0.795732,0.057622,0.305106,0.107463,0.223402
4,3882,"[38, 49]",32,3850,1559,21735,15121,409,1.626016,10.446060,22.408537,51.473301,42.853675,0.792175,0.066927,0.410272,0.123426,0.250112
5,3882,"[49, 59]",52,3830,1527,17885,18971,441,2.642276,10.391795,25.050813,61.865097,42.853675,0.775915,0.078663,0.514733,0.142844,0.279834
6,3883,"[59, 69]",84,3799,1475,14055,22801,493,4.268293,10.307684,29.319106,72.172781,42.853675,0.749492,0.094977,0.618651,0.168591,0.315144
7,3882,"[69, 79]",230,3652,1391,10256,26600,577,11.686992,9.908834,41.006098,82.081615,42.853675,0.706809,0.119430,0.721728,0.204333,0.356319
8,3882,"[79, 89]",347,3535,1161,6604,30252,807,17.632114,9.591383,58.638211,91.672998,42.853675,0.589939,0.149517,0.820816,0.238570,0.371235


AUROC , AUPR:  (0.7286720753236944, 0.20188606836738154)


 		========================== Modelo RG =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,3883,"[4.969844170281492e-08, 2.617137874376501e-05]",5,3878,1968,36856,0,0,0.254065,10.522032,0.254065,10.522032,41.354939,1.000000,0.050690,0.000000,0.096490,0.210725
1,3882,"[2.6188235817655246e-05, 0.00013900559911344303]",9,3873,1963,32978,3878,5,0.457317,10.508465,0.711382,21.030497,41.354939,0.997459,0.056180,0.105220,0.106370,0.229253
2,3882,"[0.0001390277096809918, 0.0006327969031744891]",58,3824,1954,29105,7751,14,2.947154,10.375516,3.658537,31.406013,41.354939,0.992886,0.062913,0.210305,0.118327,0.250957
3,3883,"[0.0006329976871058768, 0.0022457499453889624]",97,3786,1896,25281,11575,72,4.928862,10.272412,8.587398,41.678424,41.354939,0.963415,0.069765,0.314060,0.130108,0.270478
4,3882,"[0.002246503328889842, 0.006222325415629343]",110,3772,1799,21495,15361,169,5.589431,10.234426,14.176829,51.912850,41.354939,0.914126,0.077230,0.416784,0.142427,0.288616
5,3882,"[0.006224468396685827, 0.01591371414869624]",132,3750,1689,17723,19133,279,6.707317,10.174734,20.884146,62.087584,41.354939,0.858232,0.087008,0.519129,0.157998,0.309522
6,3883,"[0.015915776247898063, 0.040434077380432254]",194,3689,1557,13973,22883,411,9.857724,10.009225,30.741870,72.096809,41.354939,0.791159,0.100258,0.620876,0.177963,0.332664
7,3882,"[0.04045011033397045, 0.11500513836089896]",252,3630,1363,10284,26572,605,12.804878,9.849143,43.546748,81.945952,41.354939,0.692581,0.117026,0.720968,0.200220,0.349147
8,3882,"[0.11502942875820092, 0.3959130683902066]",365,3517,1111,6654,30202,857,18.546748,9.542544,62.093496,91.488496,41.354939,0.564533,0.143078,0.819460,0.228295,0.355247


AUROC , AUPR:  (0.7854317329938005, 0.17736333465552806)


 		========================== Modelo Catboost =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,3883,"[0.001203429188499736, 0.01977295623986563]",4,3879,1968,36856,0,0,0.203252,10.524745,0.203252,10.524745,50.2403,1.000000,0.050690,0.000000,0.096490,0.210725
1,3882,"[0.01977506350013166, 0.03463557583438603]",6,3876,1964,32977,3879,4,0.304878,10.516605,0.508130,21.041350,50.2403,0.997967,0.056209,0.105247,0.106424,0.229370
2,3882,"[0.034639593772539945, 0.10790547250966553]",16,3866,1958,29101,7755,10,0.813008,10.489473,1.321138,31.530823,50.2403,0.994919,0.063041,0.210414,0.118570,0.251471
3,3883,"[0.10799509414904827, 0.22273705269033897]",61,3822,1942,25235,11621,26,3.099593,10.370089,4.420732,41.900912,50.2403,0.986789,0.071457,0.315308,0.133265,0.277041
4,3882,"[0.22275444027167288, 0.2961611608842719]",79,3803,1881,21413,15443,87,4.014228,10.318537,8.434959,52.219449,50.2403,0.955793,0.080750,0.419009,0.148919,0.301771
5,3882,"[0.29621336319588726, 0.3706896924332288]",128,3754,1802,17610,19246,166,6.504065,10.185587,14.939024,62.405036,50.2403,0.915650,0.092829,0.522194,0.168569,0.330230
6,3883,"[0.37071436077691966, 0.45562923569839026]",145,3738,1674,13856,23000,294,7.367886,10.142175,22.306911,72.547211,50.2403,0.850610,0.107791,0.624050,0.191336,0.357662
7,3882,"[0.45568530503514504, 0.5614151189426936]",219,3663,1529,10118,26738,439,11.128049,9.938680,33.434959,82.485891,50.2403,0.776931,0.131278,0.725472,0.224605,0.391670
8,3882,"[0.5614500607305616, 0.7023246484671246]",344,3538,1310,6455,30401,658,17.479675,9.599522,50.914634,92.085414,50.2403,0.665650,0.168706,0.824859,0.269187,0.418878


AUROC , AUPR:  (0.8390146125725965, 0.3182167773868975)


>> **validation sample**

In [405]:
label="fraude"
quantil=10


print(f"\t\t*************************************************************************************", )
print(f"\t\t****************************** VALIDATION DATASET ************************************",)
print(f"\t\t*************************************************************************************",)

print("\n\n",f"\t\t========================= Modelo MELI =====================================")
display(utils_ml.metric_evaluation(df_validation, 'score', label, quantil))
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_validation, label, 'score'))



print("\n\n",f"\t\t========================== Modelo RG =====================================")
df_validation_2_model = transform_2_model(df_validation)
df_validation_2_model['probability'] = model.predict_proba( df_validation_2_model.drop(['fraude'], axis=1 ) )[:,1]

display( utils_ml.metric_evaluation( df_validation_2_model, 'probability', label, quantil ) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_validation_2_model, label, 'probability'))

print("\n\n",f"\t\t========================== Modelo Catboost =====================================")

df_validation_2_cat_model = transform_2_cat_model(df_validation)
df_validation_2_cat_model['probability'] = modelo_cat.predict_proba(df_validation_2_cat_model )[:,1]


display( utils_ml.metric_evaluation(df_validation_2_cat_model, 'probability', label, quantil) )
print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_validation_2_cat_model, label, 'probability'))

		*************************************************************************************
		****************************** VALIDATION DATASET ************************************
		*************************************************************************************


 		========================= Modelo MELI =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,2059,"[0, 7]",53,2006,940,19647,0,0,5.638298,10.210210,5.638298,10.210210,45.812273,1.000000,0.045660,0.000000,0.087332,0.193042
1,2059,"[7, 16]",71,1988,887,17641,2006,53,7.553191,10.118593,13.191489,20.328803,45.812273,0.943617,0.047873,0.102102,0.091124,0.198986
2,2058,"[16, 25]",91,1967,816,15653,3994,124,9.680851,10.011707,22.872340,30.340510,45.812273,0.868085,0.049548,0.203288,0.093745,0.201691
3,2059,"[25, 34]",7,2052,725,13686,5961,215,0.744681,10.444343,23.617021,40.784853,45.812273,0.771277,0.050309,0.303405,0.094456,0.199494
4,2059,"[34, 44]",11,2048,718,11634,8013,222,1.170213,10.423983,24.787234,51.208836,45.812273,0.763830,0.058128,0.407849,0.108035,0.222815
5,2058,"[44, 53]",19,2039,707,9586,10061,233,2.021277,10.378175,26.808511,61.587011,45.812273,0.752128,0.068687,0.512088,0.125879,0.251548
6,2059,"[53, 62]",29,2030,688,7547,12100,252,3.085106,10.332366,29.893617,71.919377,45.812273,0.731915,0.083546,0.615870,0.149973,0.286786
7,2058,"[62, 72]",60,1998,659,5517,14130,281,6.382979,10.169492,36.276596,82.088869,45.812273,0.701064,0.106703,0.719194,0.185216,0.331622
8,2059,"[72, 84]",128,1931,599,3519,16128,341,13.617021,9.828473,49.893617,91.917341,45.812273,0.637234,0.145459,0.820889,0.236853,0.380173


AUROC , AUPR:  (0.7357432351211651, 0.25313615106229714)


 		========================== Modelo RG =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,2059,"[3.3584951791862773e-08, 2.040149114403088e-05]",39,2020,940,19647,0,0,4.148936,10.281468,4.148936,10.281468,24.855384,1.000000,0.045660,0.000000,0.087332,0.193042
1,2059,"[2.0418949226801673e-05, 0.00010174702169646174]",48,2011,901,17627,2020,39,5.106383,10.235659,9.255319,20.517127,24.855384,0.958511,0.048629,0.102815,0.092562,0.202127
2,2058,"[0.0001017586681965407, 0.0004097261839562176]",48,2010,853,15616,4031,87,5.106383,10.230570,14.361702,30.747697,24.855384,0.907447,0.051794,0.205171,0.097995,0.210836
3,2059,"[0.00041014973443531487, 0.0013714359745061338]",57,2002,805,13606,6041,135,6.063830,10.189851,20.425532,40.937548,24.855384,0.856383,0.055860,0.307477,0.104879,0.221507
4,2059,"[0.0013729699662010967, 0.004004957937898961]",74,1985,748,11604,8043,192,7.872340,10.103324,28.297872,51.040871,24.855384,0.795745,0.060557,0.409375,0.112549,0.232125
5,2058,"[0.00400731327092175, 0.010483892191935358]",88,1970,674,9619,10028,266,9.361702,10.026976,37.659574,61.067848,24.855384,0.717021,0.065481,0.510409,0.120004,0.239806
6,2059,"[0.01049006009629321, 0.026599190891209976]",83,1976,586,7649,11998,354,8.829787,10.057515,46.489362,71.125363,24.855384,0.623404,0.071160,0.610678,0.127738,0.244268
7,2058,"[0.02661274547300362, 0.07436375508618329]",92,1966,503,5673,13974,437,9.787234,10.006617,56.276596,81.131979,24.855384,0.535106,0.081444,0.711254,0.141372,0.253120
8,2059,"[0.07439892056149801, 0.2778001307153719]",147,1912,411,3707,15940,529,15.638298,9.731766,71.914894,90.863745,24.855384,0.437234,0.099806,0.811320,0.162515,0.260853


AUROC , AUPR:  (0.673922904151898, 0.10826218267662435)


 		========================== Modelo Catboost =====================================


,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,2059,"[0.0016276316432044832, 0.020314237073166302]",37,2022,940,19647,0,0,3.936170,10.291648,3.936170,10.291648,35.556774,1.000000,0.045660,0.000000,0.087332,0.193042
1,2059,"[0.020319010216709257, 0.03421208744846859]",44,2015,903,17625,2022,37,4.680851,10.256019,8.617021,20.547666,35.556774,0.960638,0.048737,0.102916,0.092768,0.202575
2,2058,"[0.03421233998261966, 0.09402194755645214]",83,1975,859,15610,4037,81,8.829787,10.052425,17.446809,30.600092,35.556774,0.913830,0.052159,0.205477,0.098685,0.212319
3,2059,"[0.09404071041220866, 0.20605195448225383]",41,2018,776,13635,6012,164,4.361702,10.271288,21.808511,40.871380,35.556774,0.825532,0.053848,0.306001,0.101101,0.213527
4,2059,"[0.20607204928772072, 0.28013445292364114]",52,2007,735,11617,8030,205,5.531915,10.215300,27.340426,51.086680,35.556774,0.781915,0.059505,0.408714,0.110593,0.228091
5,2058,"[0.2801382740532391, 0.35122876730307817]",42,2016,683,9610,10037,257,4.468085,10.261109,31.808511,61.347788,35.556774,0.726596,0.066356,0.510867,0.121606,0.243009
6,2059,"[0.35135163104668354, 0.434352650410694]",60,1999,641,7594,12053,299,6.382979,10.174581,38.191489,71.522370,35.556774,0.681915,0.077838,0.613478,0.139728,0.267195
7,2058,"[0.43440008657347046, 0.5444937398839226]",74,1984,581,5595,14052,359,7.872340,10.098234,46.063830,81.620604,35.556774,0.618085,0.094074,0.715224,0.163294,0.292371
8,2059,"[0.544549769936535, 0.6858937691812157]",130,1929,507,3611,16036,433,13.829787,9.818293,59.893617,91.438897,35.556774,0.539362,0.123118,0.816206,0.200474,0.321782


AUROC , AUPR:  (0.7116558047409111, 0.21617459524463062)


## Using period in evaluation

> **modelo MELI**

Apresenta uma relativa estabilidade quando se olha os score para as semanas

In [346]:
df_full = pd.concat([df,df_validation])

In [406]:
## Ordenando as datas disponíveis no dataset
datas = np.sort( df_full['data'].unique() )
## total de dias para cada grupo
days = 1
## Total de grupos
total_groups = int(len(datas)/days) if int(len(datas)/days) == len(datas)/days else int(len(datas)/days+1)
## Lista de arrays contendo os dias para cada grupo
ls_group = []

i = 0
for x in range(1,total_groups+1):

    # print(x)
    group = datas[i:i+days]
    ls_group.append(group)

    i = i+days

print(f"Total de grupos:", total_groups)

Total de grupos: 45


In [407]:
# ls_group

In [408]:

label="fraude"
quantil=10

@interact

def meli_model( i  = range( len(ls_group) ) ):

    print(f"Grupo {i}: ", ls_group[i], "\n\n")


    print(f"========================== Modelo MELI =====================================")
    df_marco = df_full[df_full['data'].isin(ls_group[i])]

    display(utils_ml.metric_evaluation(df_marco, 'score', label, quantil))
    print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_marco, label, 'score'))

    print("\n\n",f"========================== Modelo RG =====================================")
    df_marco_model_dum = transform_2_model( df_full[df_full['data'].isin(ls_group[i])] )
    ######### Fitting ###########################
    df_marco_model_dum['probabilty'] = model.predict_proba( df_marco_model_dum.drop("fraude", axis=1) )[:,1]

    display(utils_ml.metric_evaluation(df_marco_model_dum, probability, label, quantil))
    print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_marco_model_dum, label, probability))


    print("\n\n",f"========================== Modelo Catboost =====================================")
    df_marco_cat = transform_2_cat_model( df_full[df_full['data'].isin(ls_group[i])] )
    ######### Fitting ###########################
    df_marco_cat['probability'] = modelo_cat.predict_proba(df_marco_cat)[:,1]

    display(utils_ml.metric_evaluation(df_marco_cat, 'probability', label, quantil))
    print( "AUROC , AUPR: ", metrics_evaluation.metrics_AUROC_AUPR(df_marco_cat, label, 'probability'))
    

interactive(children=(Dropdown(description='i', options=(0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15,…

In [38]:
!lsb_release -a

'lsb_release' n�o � reconhecido como um comando interno
ou externo, um programa oper�vel ou um arquivo em lotes.


# Requirements

In [110]:
path_root = parameters.root
!pip list --format=freeze > $path_root/requirements.txt

5697.85s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
